In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [ ]:
#Function to extract product title
def get_title(soup):
    try:
        title = soup.find('h1', attrs={'class':'productName_title'})
        title_value = title.text
        title_string = title_value.strip()
    except AttributeError:
        title_string = ''
    return title_string

#Function to get product price
def get_price(soup):
    try:
        price = soup.find('p', attrs={'class':'productPrice_price'}).text.strip()
    except AttributeError:
        price = ''
    return price
    

In [ ]:
if __name__ == '__main__':
    
    #Headers for request
    headers = ({'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36', 'Accept-Language': 'en-US, en;q=0.5'})
    root = 'https://www.dermstore.com/skin-care.list?pageNumber='
    
    page = 1
    while page != 7:
        
        #Webpage URL
        URL = root + str(page)
        print(URL)
    
        #HTTP request
        webpage = requests.get(URL, headers=headers)
    
        #Soup Object containing all data
        soup = BeautifulSoup(webpage.content, 'html.parser')
    
        #Fetch links as list
        links = soup.find_all('a', attrs={'class':'productBlock_link'})
        
        #Store links
        links_list = []
    
        #Loop for extracting links
        for link in links:
            links_list.append(link.get('href'))
        
        d = {'title':[], 'price':[]}
    
        #Loop for extracting product details from links
        for link in links_list:
            new_webpage = requests.get('https://www.dermstore.com' + link, headers=headers)
            new_soup = BeautifulSoup(new_webpage.content, 'html.parser')
        
            #Function calls to display product information
            d['title'].append(get_title(new_soup))
            d['price'].append(get_price(new_soup))
            
        page = page + 1
        
    dermstore_df = pd.DataFrame.from_dict(d)
    dermstore_df = dermstore_df.drop_duplicates(subset=None, keep='first', inplace=False, ignore_index=True)
    print(dermstore_df.head(300))


In [ ]:
dermstore_df.to_excel('dermstore.xlsx', header=True, index=False)